# BERT

In [1]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, multilabel_confusion_matrix, classification_report

import time

import warnings
warnings.filterwarnings('ignore')

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## Učitavanje podataka

In [2]:
dataset_path = 'full_dataset/full_dataset.csv'
data = pd.read_csv(dataset_path)
data

,emotion,original_text,text_tokens,preprocessed_text
0,happiness,"During the period of falling in love, each tim...","['period', 'falling', 'love', 'time', 'met', '...",period falling love time met especially met lo...
1,fear,When I was involved in a traffic accident.,"['involved', 'traffic', 'accident']",involved traffic accident
2,anger,When I was driving home after several days of...,"['driving', 'home', 'several', 'day', 'hard', ...",driving home several day hard work motorist ah...
3,sadness,When I lost the person who meant the most to me.,"['lost', 'person', 'meant']",lost person meant
4,disgust,The time I knocked a deer down - the sight of ...,"['time', 'knocked', 'deer', 'sight', 'animal',...",time knocked deer sight animal injury helpless...
...,...,...,...,...
9882,shame,He gets real humiliated and has to leave .,"['get', 'real', 'humiliated', 'leave']",get real humiliated leave
9883,shame,They aimed for higher status jobs and felt hum...,"['aimed', 'higher', 'status', 'job', 'felt', '...",aimed higher status job felt humiliated unempl...
9884,shame,He cursed his lack of self-control ; he knew t...,"['cursed', 'lack', 'selfcontrol', 'knew', 'old...",cursed lack selfcontrol knew old biddy seen fe...
9885,shame,Sometimes I've thought I 'll never forget wha...,"['sometimes', 'ive', 'thought', 'never', 'forg...",sometimes ive thought never forget happened co...


In [3]:
class_names = data['emotion'].unique()
class_names

array(['happiness', 'fear', 'anger', 'sadness', 'disgust', 'shame',
       'guilt', 'surprise'], dtype=object)

In [4]:
num_classes = len(class_names)
num_classes

8

## Priprema podataka

In [5]:
X = data['preprocessed_text'] # ili staviti original_text
Y = data['emotion']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=357)

print("Training set size = ", X_train.shape[0])
print("Test set size = ", X_test.shape[0])

Training set size =  7909
Test set size =  1978


In [6]:
encoding = {
    'happiness': 0,
    'fear': 1,
    'anger': 2,
    'sadness': 3,
    'disgust': 4,
    'shame': 5,
    'guilt': 6,
    'surprise': 7
}

y_train = [encoding[x] for x in Y_train]
y_test = [encoding[x] for x in Y_test]

Podaci se moraju posebno predprocesuirati da bi se mogli koristiti u BERT-u.

In [7]:
X_train = X_train.tolist()
X_test = X_test.tolist()

In [8]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


## Treniranje

In [9]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [10]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [11]:
learning_rate = 2e-5
epochs = 5
learner.fit_onecycle(learning_rate, epochs)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
1319/1319 [==============================] - 9168s 7s/step - loss: 1.5106 - accuracy: 0.4405 - val_loss: 0.9646 - val_accuracy: 0.6648
Epoch 2/5
1319/1319 [==============================] - 8918s 7s/step - loss: 0.8342 - accuracy: 0.7163 - val_loss: 0.8485 - val_accuracy: 0.7149
Epoch 3/5
1319/1319 [==============================] - 9066s 7s/step - loss: 0.5994 - accuracy: 0.7911 - val_loss: 0.8853 - val_accuracy: 0.7321
Epoch 4/5
1319/1319 [==============================] - 9348s 7s/step - loss: 0.3002 - accuracy: 0.8996 - val_loss: 0.9889 - val_accuracy: 0.7164
Epoch 5/5
1319/1319 [==============================] - 8905s 7s/step - loss: 0.1049 - accuracy: 0.9702 - val_loss: 1.1123 - val_accuracy: 0.7240


In [23]:
class_names = np.array(class_names).tolist()
class_names

['happiness',
 'fear',
 'anger',
 'sadness',
 'disgust',
 'shame',
 'guilt',
 'surprise']

In [24]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

62/62 [==============================] - 628s 10s/step
              precision    recall  f1-score   support

   happiness       0.84      0.84      0.84       295
        fear       0.82      0.81      0.82       313
       anger       0.68      0.65      0.67       314
     sadness       0.79      0.80      0.79       334
     disgust       0.64      0.67      0.65       226
       shame       0.61      0.63      0.62       253
       guilt       0.56      0.54      0.55       205
    surprise       1.00      1.00      1.00        38

    accuracy                           0.72      1978
   macro avg       0.74      0.74      0.74      1978
weighted avg       0.72      0.72      0.72      1978



array([[247,   9,   4,  19,   5,   4,   7,   0],
       [ 10, 253,  16,   8,  12,   8,   6,   0],
       [  5,  10, 204,  15,  32,  24,  24,   0],
       [ 12,  11,   7, 267,  11,  12,  14,   0],
       [  2,   7,  29,   9, 152,  19,   8,   0],
       [ 12,   9,  17,   8,  17, 160,  30,   0],
       [  5,   8,  22,  13,  10,  36, 111,   0],
       [  0,   0,   0,   0,   0,   0,   0,  38]], dtype=int64)

## Testiranje

In [25]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['happiness',
 'fear',
 'anger',
 'sadness',
 'disgust',
 'shame',
 'guilt',
 'surprise']

In [29]:
message = 'I like apples'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: happiness (0.32)


## Spremanje modela

In [19]:
predictor.save("models/bert_model")